## The Context Manager Protocol

We use the builtin `with ...:` for Context Manager.

- Classes that implement the `__enter__(self)` and `__exit__(self, exc_type, exc_val, exc_tb)` methods can be used inside the `with` expression.
- `__enter__` is used for setup, and its returned value will be **bound to** the `as` **target**.
- `__exit__` will always run when the indented block under `with` finishes, regardless of whether an exception was raised or not. If it returns `True`, it suppresses exceptions otherwise, the raised exception will propagate.
- Exception information is passed to the `__exit__` method through positional parameters.
- Custom classes can implement these methods for tailored resource management.

In [21]:
class DemoCM:
    def __enter__(self):
        print("Entering")
        return "resource"

    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"Exiting, exception={exc_type}")
        return False; # returning False re-raises any exception

try:
    with DemoCM() as res:
        print(f"Inside 'with', got: {res}") # returned value of '__enter__' will be bound to 'res' in this case
        # As we exit this `with` block, control passes to __exit__ and is executed
        
        #raise RuntimeError("Test Error")
except Exception as e:
    print(f"Caught outside 'with': {e}")
    

Entering
Inside 'with', got: resource
Exiting, exception=None


## Custom Resource Management: Writing Context Managers
- Whenever you need custom setup/teardown logic, you can write your own Context Manager.
- A context manager ensures that teardown always runs, even if errors occur in the block.  
- Two approaches: implement `__enter__`/`__exit__` in a class or use the simpler generator-based decorator.  

In [27]:
class MyContextManager:
    def __init__(self, timeout):
        self.timeout = timeout

    def __enter__(self):
        print("Setup complete")
        return "a simple value"

    def __exit__(self, exc_type, exc_value, exc_traceback):
        print(f"Teardown")
        print(f"Exception type: {exc_type}")
        print(f"Exception value: {exc_value}")
        print(f"Exception traceback: {exc_traceback}")
        return True # returning True supresses the Exception

with MyContextManager(timeout=30) as cm:
    print(cm)
    print("Inside the block")
    raise ValueError("Simulated problem")
    # As we exit this `with` block, if there is any exception, the exception info is passed to __exit__ method

Setup complete
a simple value
Inside the block
Teardown
Exception type: <class 'ValueError'>
Exception value: Simulated problem
Exception traceback: <traceback object at 0x10abdef80>


## The `@contextlib.contextmanager` Decorator
- Provided by the `contextlib` module to turn a generator into a context manager.  
- Decorated function needs exactly one `yield`.  
- Code before `yield` runs as `__enter__`; code after (or in `finally`) runs as `__exit__`.  
- Simplifies many common patterns without writing a full class.

###  Generator Structure for `@contextmanager`
- Wrap the `yield` in `try...finally` to ensure teardown even on errors.  
- The value yielded is bound to `as var` in the `with` statement (if used).  
- You can catch exceptions inside the generator if you want to suppress them.

In [ ]:
import os
from contextlib import contextmanager

@contextmanager
def change_directory(destination):
    """
    Temporarily switch into destination. If the directory does not exist,
    it is created just before the switch.

    Args:
        destination (str): Path to the directory that should become the working directory
    """

    origin_dir = os.getcwd()

    try:
        print(f"Changing into {destination}")
        os.makedirs(destination, exist_ok=True) # do not raise if dir already exist
        os.chdir(destination)
        yield os.getcwd()
    finally:
        print(f"Reverting to original dir: {origin_dir}")
        os.chdir(origin_dir)

print(f"Start: {os.getcwd()}")

with change_directory("temp_dir") as new_dir:
    print(f"Inside: {new_dir}")

print(f"End: {os.getcwd()}")

Start: /Users/lauromueller/Documents/courses/python-devops/code/error-handling
Changing into temp_dir
Inside: /Users/lauromueller/Documents/courses/python-devops/code/error-handling/temp_dir
Reverting to original dir: /Users/lauromueller/Documents/courses/python-devops/code/error-handling
End: /Users/lauromueller/Documents/courses/python-devops/code/error-handling
